# LARCV file making for DUNE

## I. Introduction and Workflow

<div>
    <br>
    <img src="assets/sbn.png"/>
    <figcaption style="text-align: center; font-size: 24px;">ICARUS (left) and SBND/DUNE (right) software chain</figcaption>
</div>

## II. Setup
LArSoft is really difficult to setup outside of the fermilab computers...
- ssh into a fermilab gpvm (i.e. `dunegpvm04.fnal.gov`)
- Copy custom config fcls `cp /exp/dune/data/users/nivedpm/ML/spine_workshop/pdvd/*.fcl /path/to/your/work/area`
- If you need apptainer run `source /exp/dune/data/users/nivedpm/ML/spine_workshop/apptainer.sh` then `app`

Option 1 (recommended) - use setup from ups
- Setup ups `source /cvmfs/dune.opensciencegrid.org/products/dune/setup_dune.sh`
- Setup dunesw`setup dunesw v10_17_00d00 -q e26:prof #or latest version should work too`

Option 2 - use my custom area
- Copy setup script `/exp/dune/data/users/nivedpm/ML/spine_workshop/setup.sh /path/to/your/work/area`
- Run setup script `source setup.sh` #sets up DUNE stack and Supera config


## III. Run

We will be running using `dunesw v10_17_00`. 

### Workflows
I have included ProtoDUNE-VD, ProtoDUNE-HD, ProtoDUNE-SP, and DUNE-FD workflows in `/exp/dune/data/users/nivedpm/ML/spine_workshop/`. The corresponding directories are `pdvd`,`pdhd`,`pdsp` and `dunefd`.

Each directory contains a `launch_all.sh` script, which can be used to run the full simulation chain for the respective detector, including the required truth information.

We will also take a look at the corresponding Supera configuration files.


### Run simulation (optional) - Example PDVD
You can run the production simulation like this, but we will focus on running supera. The full workflow can be run as below
```shell
PREFIX=pdvd_mpvmpr

lar -c gen_protodunevd_mpvmpr.fcl  -n 1  -o ${PREFIX}_gen.root  -T ${PREFIX}_gen_hist.root
lar -c protodunevd_g4_stage1_simplemerge.fcl  -o ${PREFIX}_g4_stage1.root -T ${PREFIX}_g4_stage1_hist.root ${PREFIX}_gen.root
lar -c protodunevd_g4_stage2_mcreco.fcl -o ${PREFIX}_g4_stage2.root -T ${PREFIX}_g4_stage2_hist.root ${PREFIX}_g4_stage1.root
lar -c protodunevd_detsim_nopds.fcl  -o ${PREFIX}_detsim.root  -T ${PREFIX}_detsim_hist.root    ${PREFIX}_g4_stage2.root
lar -c protodunevd_reco_cluster3d.fcl -o ${PREFIX}_reco.root -T ${PREFIX}_reco_hist.root      ${PREFIX}_detsim.root
lar -c run_supera_pdunevd_mpvmpr.fcl ${PREFIX}_reco.root

```

Takes ~1-2 min to run and process a mpvmpr event using the workflow above.

### Run supera 
The path to the `reco` data is at `/exp/dune/data/users/nivedpm/ML/spine_workshop/pdvd` on any fermilab machine. If you didn't run the steps above 
```shell
cp /exp/dune/data/users/nivedpm/ML/spine_workshop/pdvd/pdvd_mpvmpr_reco.root /path/to/your/work/area #copy existing reco file to work area
lar -c run_supera_pdunevd_mpvmpr.fcl -s pdvd_mpvmpr_reco.root #supera label making
```

This will produce a `larcv.root` file, who's contents will be covered in this workshop [here](https://github.com/DeepLearnPhysics/spine-workshop-2026/blob/main/basics/larcv/LArCV_visualization.ipynb)


### Run supera for data 
An example ProtoDUNE-VD beam data reco file has been placed at `/exp/dune/data/users/nivedpm/ML/spine_workshop/pdvd`
You can run the following to get the `larcv_data.root` file
```shell
cp /exp/dune/data/users/nivedpm/ML/spine_workshop/pdvd/np02vd_raw_run039433_0000_df-s05-d3_dw_0_20250917T153840_reco_stage0_20260116T135116_beam.root /path/to/your/work/area #copy existing reco file to work area
lar -c run_supera_data.fcl np02vd_raw_run039433_0000_df-s05-d3_dw_0_20250917T153840_reco_stage0_20260116T135116_beam.root #supera label making
```
Everything is available at NERSC as well at `/global/cfs/cdirs/m5252/nivedpm/ML/spine_workshop`

### Explore tools (recommended)
To investigate what's in a `.fcl` file use 
```shell
fhicl-dump *.fcl
```
To investigate what's in an art root file use
```shell
lar -c eventdump.fcl -s file.root -n 1
```

![image.png](assets/goodjob.jpeg)